In [19]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

22/09/11 10:59:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/11 10:59:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [20]:
retailDataDay = "data/retail-data/by-day/"
retailDataDaySmall = "data/retail-data/by-day/2011-12-06.csv"

### Note remove all the data from data/retail-data/by-day/ and keep only 1 csv file - 2011-12-06.csv
 - Need to remove manually

In [21]:
staticDataFrame = spark\
.read\
.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load(retailDataDaySmall)

In [22]:
staticSchema = staticDataFrame.schema

In [23]:
# maxFilesPerTrigger option simply specifies the number of files we should read in at once
# This is to make our demonstration more “streaming,” and in a production scenario this would probably be omitted

streamingDataFrame = spark\
.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load(retailDataDay)

In [24]:
from pyspark.sql.functions import col, window

In [25]:
purchaseByCustomer = streamingDataFrame\
.selectExpr(\
           "CustomerId",
           "(UnitPrice * Quantity) as total_cost",\
           "InvoiceDate")\
.groupBy(\
        col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [26]:
purchaseByCustomer.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()

22/09/11 10:59:44 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c947e4d4-5ff2-43ca-8943-9e2c5732f8ae. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [27]:
spark.sql("""
select *
from customer_purchases
order by 'sum(total_cost)' desc""")\
.count()

0

In [28]:
purchaseByCustomer.writeStream.trigger(processingTime='10 seconds')\
  .format("console").outputMode("complete").start()

22/09/11 10:59:46 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e05ed3bf-aaa3-4e32-a20c-6d0fef4c19ac. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----------+--------------------+-------------------+
|CustomerId|              window|    sum(total_cost)|
+----------+--------------------+-------------------+
|   14071.0|[2011-12-06 00:00...|             154.58|
|   14543.0|[2011-12-06 00:00...|             -32.05|
|   12417.0|[2011-12-06 00:00...|             369.73|
|   14503.0|[2011-12-06 00:00...|             337.75|
|   18005.0|[2011-12-06 00:00...|             109.75|
|   15189.0|[2011-12-06 00:00...|             348.72|
|   15622.0|[2011-12-06 00:00...|-381.95000000000005|
|   12839.0|[2011-12-06 00:00...|               -4.5|
|   17861.0|[2011-12-06 00:00...| 438.58000000000015|
|   17389.0|[2011-12-06 00:00...|            5134.88|
|   17490.0|[2011-12-06 00:00...|              -40.4|
|   16133.0|[2011-12-06 00:00...| 309.21999999999997|
|   13362.0|[2011-12-06 00:00...|              -5.25|
|   17841.0|[2011-12-06 00:00...|  974.

22/09/11 11:00:20 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 33800 milliseconds
22/09/11 11:01:07 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 27603 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+----------+--------------------+-------------------+
|CustomerId|              window|    sum(total_cost)|
+----------+--------------------+-------------------+
|   15136.0|[2011-12-07 00:00...|              256.9|
|   13743.0|[2011-12-07 00:00...|             275.76|
|   14071.0|[2011-12-06 00:00...|             154.58|
|   14543.0|[2011-12-06 00:00...|             -32.05|
|   15525.0|[2011-12-07 00:00...| 299.78999999999996|
|   15568.0|[2011-12-07 00:00...| 335.77000000000004|
|   12826.0|[2011-12-07 00:00...|              142.6|
|   12417.0|[2011-12-06 00:00...|             369.73|
|   17490.0|[2011-12-07 00:00...|-42.760000000000005|
|   14503.0|[2011-12-06 00:00...|             337.75|
|   13756.0|[2011-12-07 00:00...|  846.3999999999999|
|   12912.0|[2011-12-07 00:00...| 187.92000000000002|
|   18005.0|[2011-12-06 00:00...|             109.75|
|   15189.0|[2011-12-06 00:00...|      

22/09/11 11:01:50 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 20907 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+----------+--------------------+-------------------+
|CustomerId|              window|    sum(total_cost)|
+----------+--------------------+-------------------+
|   15136.0|[2011-12-07 00:00...|              256.9|
|   13743.0|[2011-12-07 00:00...|             275.76|
|   17144.0|[2011-12-08 00:00...| 388.68000000000006|
|   14071.0|[2011-12-06 00:00...|             154.58|
|   14543.0|[2011-12-06 00:00...|             -32.05|
|   13408.0|[2011-12-08 00:00...| 445.84000000000003|
|   15525.0|[2011-12-07 00:00...| 299.78999999999996|
|   16891.0|[2011-12-08 00:00...|             115.95|
|   15568.0|[2011-12-07 00:00...| 335.77000000000004|
|   12826.0|[2011-12-07 00:00...|              142.6|
|   12417.0|[2011-12-06 00:00...|             369.73|
|   13756.0|[2011-12-07 00:00...|  846.3999999999999|
|   17490.0|[2011-12-07 00:00...|-42.760000000000005|
|   14503.0|[2011-12-06 00:00...|      

22/09/11 11:02:05 ERROR MicroBatchExecution: Query customer_purchases [id = 44e0dc9b-b440-42c5-abec-a352becbbc17, runId = 50e7b20a-b5ba-4b33-b82c-b9b92a50100a] terminated with error
org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(CustomerID#12513, window#58, 200), true, [id=#808]
+- *(1) HashAggregate(keys=[knownfloatingpointnormalized(normalizenanandzero(CustomerID#12513)) AS CustomerID#12513, window#58], functions=[partial_sum(total_cost#48)], output=[CustomerID#12513, window#58, sum#88])
   +- *(1) Project [named_struct(start, precisetimestampconversion(((((CASE WHEN (cast(CEIL((cast((precisetimestampconversion(cast(InvoiceDate#12511 as timestamp), TimestampType, LongType) - 0) as double) / 8.64E10)) as double) = (cast((precisetimestampconversion(cast(InvoiceDate#12511 as timestamp), TimestampType, LongType) - 0) as double) / 8.64E10)) THEN (CEIL((cast((precisetimestampconversion(cast(InvoiceDate#12511 as timestamp), Timestamp

-------------------------------------------
Batch: 3
-------------------------------------------
+----------+--------------------+-------------------+
|CustomerId|              window|    sum(total_cost)|
+----------+--------------------+-------------------+
|   15136.0|[2011-12-07 00:00...|              256.9|
|   13743.0|[2011-12-07 00:00...|             275.76|
|   17114.0|[2011-12-04 00:00...|  90.89000000000001|
|   17144.0|[2011-12-08 00:00...| 388.68000000000006|
|   14071.0|[2011-12-06 00:00...|             154.58|
|   14543.0|[2011-12-06 00:00...|             -32.05|
|   13408.0|[2011-12-08 00:00...| 445.84000000000003|
|   15525.0|[2011-12-07 00:00...| 299.78999999999996|
|      null|[2011-12-04 00:00...|  4245.470000000001|
|   16891.0|[2011-12-08 00:00...|             115.95|
|   15568.0|[2011-12-07 00:00...| 335.77000000000004|
|   16988.0|[2011-12-04 00:00...|             126.12|
|   12826.0|[2011-12-07 00:00...|              142.6|
|   12417.0|[2011-12-06 00:00...|      

22/09/11 11:02:28 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 18012 milliseconds
22/09/11 11:03:57 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 17418 milliseconds


-------------------------------------------
Batch: 4
-------------------------------------------
+----------+--------------------+-------------------+
|CustomerId|              window|    sum(total_cost)|
+----------+--------------------+-------------------+
|   15136.0|[2011-12-07 00:00...|              256.9|
|   13743.0|[2011-12-07 00:00...|             275.76|
|   17114.0|[2011-12-04 00:00...|  90.89000000000001|
|   17144.0|[2011-12-08 00:00...| 388.68000000000006|
|   14071.0|[2011-12-06 00:00...|             154.58|
|   14543.0|[2011-12-06 00:00...|             -32.05|
|   13408.0|[2011-12-08 00:00...| 445.84000000000003|
|      null|[2011-12-04 00:00...|  4245.470000000001|
|   15525.0|[2011-12-07 00:00...| 299.78999999999996|
|   16891.0|[2011-12-08 00:00...|             115.95|
|   16988.0|[2011-12-04 00:00...|             126.12|
|   12826.0|[2011-12-07 00:00...|              142.6|
|   15568.0|[2011-12-07 00:00...| 335.77000000000004|
|   12417.0|[2011-12-06 00:00...|      

### Once the files are processed, copy 1 CSV file each time manually to data/retail-data/by-day/
- Notice that processing is again started

In [29]:
spark.stop()